## File names here:

In [1]:
tissues_with_atac = ['CHO','Brain','Liver','Lung','Kidney','Spleen','BMDMwt','BMDM1hKLA']

## Parameters specific to where your folders are and your data
parameter_file = 'params/params.yaml'
import yaml
import sys

with open(parameter_file,'r') as f:
    doc = yaml.load(f)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
annotation=doc['annotation']
tss_annotation = doc['tss_annotation']
mRNA_peak_file = doc["mRNA_peak_file"]

In [2]:
mRNA_peak_file

'/data/isshamie/genome/ncbi_anno_103/mRNA_final.peak'

In [2]:
ls {tss_annotation}

/data/isshamie/genome/ncbi_anno_103/start_site_mRNA_final.tsv


In [3]:
from create_output import *

txn_f = "Results/tss_annotation/txn_df_02_tissues.p"
#expr_f = "Results/merged/samples.merge.peaksexpression.log10"
expr_f = "Results/merged/tissues.merge.peaksexpression.log10"
anno_f = "Results/tss_annotation_peaks/all_peaks_txn_df_02_maxinfo.tsv"
rank_func = rank_median_samples
atac_f = "Results/ATAC_results/CHO/ATAC_ppr.naive_overlap.narrowPeak.sort"
tss_f = mRNA_peak_file
all_atac = "Results/ATAC_results/ATAC_merge.bed"



In [4]:
import os
import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm
### notebook specific configuration ###
from os.path import basename
#mpl.style.use('ggplot')
mpl.style.use('fivethirtyeight')
from cycler import cycler
mpl.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')

from Homer import *

print('Number of tissues: ',len(tissues))

('Number of tissues: ', 13)


In [5]:
if not os.path.exists("Results/output"):
    os.mkdir("Results/output")

In [6]:
%%time
# bed_df, meta_df = construct_peakID(txn_anno_f, expr_f, anno_f, rank_func,
#                      atac_f=None, col_name=None)

bed_df, meta_df = construct_all_peaks(txn_f, expr_f, anno_f, rank_func, tss_f, atac_f,
                        all_atac=all_atac, out_f="Results/output/TSS1")

Constructing TSS not observed experimentally


15042it [02:52, 87.28it/s]


Constructing TSS observed experimentally
Including ATAC information


45817it [06:34, 116.17it/s]
45817it [18:05, 42.19it/s]


CPU times: user 37min 46s, sys: 8.91 s, total: 37min 55s
Wall time: 37min 36s


In [7]:
bed_df.head()

,Chr,Start,End,ID,Stat,Strand
p2@Bmp4_XM_027386529.1,NW_020822366.1,721599,721750,p2@Bmp4_XM_027386529.1,0.506505,-
p3@Bmp4_XM_027386529.1,NW_020822366.1,721832,721983,p3@Bmp4_XM_027386529.1,0.580925,-
p1@Bmp4_XM_027386529.1,NW_020822366.1,722208,722359,p1@Bmp4_XM_027386529.1,1.779596,-
p1@Bmp4_XM_027386528.1,NW_020822366.1,724533,724684,p1@Bmp4_XM_027386528.1,4.074290,-
p1@Cdkn3_XM_027386531.1,NW_020822366.1,1085431,1085582,p1@Cdkn3_XM_027386531.1,1.212188,+


In [8]:
meta_df.head()

,Tissues,cs,CHO ATAC Region,ID,Gene,Gene ID,Transcript,Is Experimental,has ATAC
p2@Bmp4_XM_027386529.1,"Spleen,MiscOrgans,FemaleReproductive,Brain,Lun...",0b00,,90264,Bmp4,,XM_027386529.1,1,0
p3@Bmp4_XM_027386529.1,"Spleen,MiscOrgans,FemaleReproductive,Brain,Lun...",0b00,,181215,Bmp4,,XM_027386529.1,1,1
p1@Bmp4_XM_027386529.1,"Spleen,MiscOrgans,FemaleReproductive,Brain,Lun...",0b11,,348823,Bmp4,,XM_027386529.1,1,1
p1@Bmp4_XM_027386528.1,"Heart,Spleen,MiscOrgans,FemaleReproductive,Bra...",0b11,,308710,Bmp4,,XM_027386528.1,1,1
p1@Cdkn3_XM_027386531.1,"BMDMwt,Spleen,MiscOrgans,FemaleReproductive,CH...",0b11,,237253,Cdkn3,,XM_027386531.1,1,0


-----------------------------

-----------------------------